## Решение задачи прогнозирования задолженности по услугам ЖКХ с использованием полностью гомоморфного шифрования

В настоящее время большинство современных компаний используют в своей деятельности машинное обучение, которое позволяет решать различные задачи: от прогнозирования задолженности до создания рекомендательных систем. При этом машинное обучение требовательно к вычислительной мощности, поэтому вместо закупки дорогостоящего оборудования возможно использование арендованного облачного сервера или специального сервиса, например, Yandex DataSphere. Но так как данные, использующиеся для обучения модели, передаются на сторонний сервер, возникает проблема обеспечения их безопасности, решить которую позволяет использование полностью гомоморфного шифрования, позволяющего выполнять вычисления над зашифрованными данными, не раскрывая их содержание.  

В данной работе демонстрируется применение полностью гомоморфного шифрования для машинного обучения на примере решения задачи кредитного скоринга с предварительным шифрованием данных на стороне клиента (этап 1) и обучением и оценкой модели на зашифрованных данных на стороне сервера (этап 2).
Также проводится исследование влияние размера данных на производительность полностью гомоморфного шифрования (этап 2а)

В качестве библиотеки полностью гомоморфного шифрования выбрана TenSEAL, основанная на Microsoft SEAL, которая в настоящее время является наиболее используемой библиотекой полностью гомоморфного шифрования. С помощью API она обеспечивает простоту использования языка Python, при этом сохраняя эффективность за счет реализации большинства операций с использованием C++.

### Этап 1 - Шифрование данных (клиент)

**Импорт библиотек**

In [ ]:
import pandas as pd
import tenseal as ts
from fhe_cs import encrypt_data
from sklearn.model_selection import train_test_split

#### 1 Подготовка данных

Набор уже подготовленных и предобработанных данных не может быть опубликован в открытом доступе по соглашению и содержит сведения о 4400 жителях, использующих цифровой сервис для расчетов в сфере ЖКХ, и включает 23 признака, такие как, например, тип жилья, сезонный фактор, подключенные уведомления или автоплатёж.

In [2]:
df = pd.read_csv('data.csv', low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Has_Automatic_Payments_Set     4400 non-null   float64
 1   Employment_Status              4400 non-null   float64
 2   Income_Level                   4400 non-null   float64
 3   Family_Size                    4400 non-null   float64
 4   Average_Payment_Delay_Days     4400 non-null   float64
 5   Payment_Method                 4400 non-null   float64
 6   Seasonal_Factors               4400 non-null   float64
 7   Frequency_of_Mobile_App_Usage  4400 non-null   float64
 8   Number_of_Children             4400 non-null   float64
 9   Payment_History_1M             4400 non-null   float64
 10  Payment_History_3M             4400 non-null   float64
 11  Payment_History_6M             4400 non-null   float64
 12  Total_Debt                     4400 non-null   f

Разделим данные на обучающую и тестовую выборки

In [ ]:
df = df.sample(frac=0.5)
x = df.drop(columns=['Is_Delinquent'])
y = df['Is_Delinquent'].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Создадим тензоры.

In [ ]:
x_train = torch.tensor(x_train.values).to(torch.float32)
x_test = torch.tensor(x_test.values).to(torch.float32)
y_train = torch.tensor(y_train.values).to(torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values).to(torch.float32).unsqueeze(1)

#### 2 Шифрование данных

Для шифрования данных предлагается использовать схему CKKS, так как она позволяет работать с вещественными числами, которыми представлены данные. 
Параметры шифрования схемы CKKS включают в себя:
* степень полиномиального модуля;
* размеры модуля коэффициента.

Определим значения параметров шифрования, для чего сначала определим количество необходимых операций умножения:
* для операции скалярного произведения - 1;
* для аппроксимации сигмоидной функции - 2;
* для осуществления обратного распространения ошибки - 3.  

Таким образом, мультипликативная глубина равна 1 + 2 + 3 = 6, что определяет количество чисел, состовляющих модуль коэффцииента. 

Степень полиномиального модуля poly_mod_grade, которую ещё называют коэффициентом масштабирования, должна быть степенью двойки (1024, 2048, 4096, 8192 и т. д.), так как она определяет точность кодирования для двоичного представления числа, то есть контролирует точность дробной части, поскольку это значение, на которое умножаются открытые тексты перед кодированием в полином целочисленных коэффициентов.  
Приняв желаемый уровнь защищенности, эквивалентный AES, равный 128 битам, степень полиномиального модуля должна быть не ниже 8192, что позволит группировать до 4096 значений в одном зашифрованном тексте.

Размеры модуля коэффициента coeff_mod_bit_sizes представляют собой список двоичных размеров, используя который TenSEAL генерирует список простых чисел этих двоичных размеров. Так как желаемый уровень защищённости составляет 128 бит, количество умножений равно 6, а 128 / 6 = 21.3, то в качестве двоичного размера следует использовать 21 бит. При этом двоичные размеры в количестве, равном количеству умножений, определяются в coeff_mod_bit_sizes[1:-1] и должны быть равными друг другу, а первый и последний размер в данном списке должны быть больше их по размеру.  
Таким образом, размеры модуля коэффициента [40, 21, 21, 21, 21, 21, 21, 40] обозначают то, что модуль коэффициента будет содержать 8 простых чисел: первое и последнее по 40 бит и остальные по 21 бит. 

In [3]:
poly_mod_grade = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]

Создадим TenSEALContext для указания схемы и параметров. По умолчанию создаются ключи релинеаризации, а также влючена автоматическая релинеаризация и масштабирование.

In [4]:
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_grade, -1, coeff_mod_bit_sizes)

Также укажем параметр global_scale, который используется в качестве значения масштаба по умолчанию, и который равен 21 биту.

In [5]:
ctx_training.global_scale = 2 ** 21

Создадим ключи Галуа для выполнения операции скалярного произведения.

In [6]:
ctx_training.generate_galois_keys()

Сохраним созданый TenSEALContext для использования его на сервере.

In [7]:
with open('encrypted data//context.hex', 'wb') as file:
    file.write(ctx_training.serialize(save_secret_key=True))

Для обеспечения безопасности данных на сервер нужно загружать уже зашифрованные данные, для чего используем функцию encrypt_data из ранее разработанного модуля fhe_cs. 


In [10]:
encrypt_data(x_train, 'x_train', ctx_training)

Шифрование заняло 45 секунд


In [11]:
encrypt_data(y_train, 'y_train', ctx_training)

Шифрование заняло 48 секунд


In [12]:
encrypt_data(x_test, 'x_test', ctx_training)

Шифрование заняло 10 секунд


In [13]:
encrypt_data(y_test, 'y_test', ctx_training)

Шифрование заняло 11 секунд


Таким образом, подготовленные данные были зашифрованы в отдельные файлы для их дальнейшей передачи на сервер.